In [1]:
import numpy as np
from sklearn import datasets, model_selection, svm, metrics
from sklearn.datasets import fetch_mldata
from sklearn.neural_network import MLPClassifier
# URL:https://qiita.com/nvtomo1029/items/200dbfe5928c4a0cd625

In [2]:
mnist = datasets.fetch_mldata('MNIST original', data_home='data/src/download/')

print(type(mnist))
print(mnist.keys())

C:\Users\Owner\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:85: DeprecationWarning: Function fetch_mldata is deprecated; fetch_mldata was deprecated in version 0.20 and will be removed in version 0.22. Please use fetch_openml.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\Owner\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:85: DeprecationWarning: Function mldata_filename is deprecated; mldata_filename was deprecated in version 0.20 and will be removed in version 0.22. Please use fetch_openml.
  warnings.warn(msg, category=DeprecationWarning)


<class 'sklearn.utils.Bunch'>
dict_keys(['DESCR', 'COL_NAMES', 'target', 'data'])


In [3]:
mnist_data = mnist.data / 255 # 8bitのデータを0.0～1.0に正規化
mnist_label = mnist.target

## Data split

In [4]:
data_train, data_test, label_train, label_test = model_selection.train_test_split(mnist_data, mnist_label, test_size=0.05, train_size=0.3) # 学習用は全データの30%、テスト用は5%に設定
print('train_size:', data_train.shape[0]) # 学習データ数
print('test_size:', data_test.shape[0]) # テストデータ数

train_size: 21000
test_size: 3500


In [5]:
classifiers = [
    (('Adam_1layer', MLPClassifier(solver="adam", early_stopping=True))), 
    (('Adam_2layer)', MLPClassifier(solver="adam", early_stopping=True, hidden_layer_sizes=(100, 10)))), 
    (('Adam_3layer)', MLPClassifier(solver="adam",  early_stopping=True, hidden_layer_sizes=(100, 100, 10)))), 
    (('Adam_4layer)', MLPClassifier(solver="adam",  early_stopping=True, hidden_layer_sizes=(100, 100, 100, 10)))) 
]

In [6]:
classifier_names = [
    'Adam', 
    'Adam_2layer', 
    'Adam_3layer', 
    'Adam_4layer' 
]

In [8]:
for name, clf in classifiers:
    print(name, 'start')
    # 予測
    clf.fit(data_train, label_train)
    pre = clf.predict(data_test)
    ### 評価 ###
    # accuracy
    ac_score = metrics.accuracy_score(label_test, pre)
    
    # confusion_matrix
    co_mat = metrics.confusion_matrix(label_test, pre)
    
    # precision, recall, f1-score の評価
    cl_repo = metrics.classification_report(label_test, pre)
    
    print('AC Score:', ac_score)
    print('confusion matrix:')
    print(co_mat)
    print('classification report:')
    print(cl_repo)
    
    print(name, 'end')

Adam_1layer start
AC Score: 0.9617142857142857
confusion matrix:
[[339   0   1   0   1   0   2   0   6   0]
 [  0 408   3   1   0   0   1   2   1   0]
 [  2   0 342   1   3   0   1   3   2   0]
 [  0   1   6 328   0   2   2   2   4   0]
 [  0   2   1   0 303   0   2   1   3   6]
 [  1   1   1   6   2 318   1   1   2   1]
 [  0   0   1   0   2   3 328   0   1   0]
 [  0   3   4   0   2   2   0 346   0   6]
 [  0   2   2   1   1   5   0   0 333   1]
 [  1   1   1   2   6   2   0   5   2 321]]
classification report:
              precision    recall  f1-score   support

         0.0       0.99      0.97      0.98       349
         1.0       0.98      0.98      0.98       416
         2.0       0.94      0.97      0.96       354
         3.0       0.97      0.95      0.96       345
         4.0       0.95      0.95      0.95       318
         5.0       0.96      0.95      0.95       334
         6.0       0.97      0.98      0.98       335
         7.0       0.96      0.95      0.96     

## parameter

In [9]:
# ハイパーパラメータのリスト
tuned_parameters = [
    {
        # 最適化手法
        "solver":["lbfgs", "sgd", "adam"], 
        # 隠れ層の層の数と、各層のニューロンの数
        "hidden_layer_sizes":[(100,), (100, 10), (100, 100, 10), (100, 100, 100, 10)], 
    }
]

# ニューラルネットワークの分類器とハイパーパラメータのリストを定義
licv=model_selection.GridSearchCV(MLPClassifier(early_stopping=True), param_grid=tuned_parameters, scoring="accuracy")

In [10]:
# 最適なハイパーパラメータを探索する
licv.fit(data_train, label_train)

# 最適なパラメータを表示(リストで指定している項目のみ)
print('licv.best_params_ : ', licv.best_params_)

# 最適なパラメータを表示(リストで指定していない項目も含め全項目表示)
print('licv.best_estimator_ : ', licv.best_estimator_)

# 最適なパラメータ指定時の評価値
print('licv.best_score_ : ', licv.best_score_)

C:\Users\Owner\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


licv.best_params_ :  {'hidden_layer_sizes': (100,), 'solver': 'lbfgs'}
licv.best_estimator_ :  MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=True, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)
licv.best_score_ :  0.9570952380952381


In [11]:
# 求められた最適なパラメータで予測、評価
pre = licv.predict(data_test)
# 正解率取得
ac_score = metrics.accuracy_score(label_test, pre)
print('AC Score:', ac_score)
# 混同行列の出力
co_mat = metrics.confusion_matrix(label_test, pre)
print('confusion matrix:')
print(co_mat)
# precision, recall, f1-score の評価
cl_repo = metrics.classification_report(label_test, pre)
print('classification report:')
print(cl_repo)

AC Score: 0.9571428571428572
confusion matrix:
[[342   0   0   0   1   0   2   0   3   1]
 [  0 408   3   1   1   0   1   1   0   1]
 [  2   0 335   6   1   1   1   2   4   2]
 [  1   0   9 323   0   2   1   3   4   2]
 [  0   0   1   1 304   0   1   3   3   5]
 [  1   1   1   6   1 313   4   1   2   4]
 [  1   0   2   0   2   3 325   0   2   0]
 [  0   3   4   1   1   0   0 346   0   8]
 [  1   2   2   5   0   3   0   0 332   0]
 [  1   1   1   3   2   2   0   9   0 322]]
classification report:
              precision    recall  f1-score   support

         0.0       0.98      0.98      0.98       349
         1.0       0.98      0.98      0.98       416
         2.0       0.94      0.95      0.94       354
         3.0       0.93      0.94      0.93       345
         4.0       0.97      0.96      0.96       318
         5.0       0.97      0.94      0.95       334
         6.0       0.97      0.97      0.97       335
         7.0       0.95      0.95      0.95       363
         8.0